# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Известно, что каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. В таблице дано четыре столбца типа *float64* и один столбец "is_ultra" типа *int64*, всего строк - 3214:
- сalls — количество звонков;
- minutes — суммарная длительность звонков в минутах;
- messages — количество sms-сообщений;
- mb_used — израсходованный интернет-трафик в Мб;
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

В данном шаге необходимо разделить данные на три выборки: обучающую, валидационную и тестовую в размере 60%, 20% и 20%. Для этого дважды используем функцию *train_test_split*.

In [ ]:
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=12345)

df_train, df_test = train_test_split(df_train, test_size=0.25, random_state=12345) # 0.25 x 0.8 = 0.2

Сначала разделили данные на обучающую и валидационную выборки в размере 80% и 20%. Затем от обучающей отделили 25% на тестовую выборку, чтобы итоговый результат был 20% от всех данных.

## Исследуйте модели

Перед нами задача классификации, нужно построить модель, которая определит подходящий тариф. Для этого исследуем три модели: дерево решений, случайный лес и логистическую регрессию.

Начнем с дерева решений, для максимально высокого качества предсказаний переберем в алгоритме разные варианты глубины дерева и найдем максимальный accuracy для валидационной выборки. 

In [ ]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [ ]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [ ]:
best_model = None
best_result = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        
print("Accuracy лучшей модели на валидационной выборке:", best_result)
print("Depth:",depth)

Accuracy лучшей модели на валидационной выборке: 0.7884914463452566
Depth: 5


Рассмотрим следующую модель - случайный лес. Будем изменять гиперпараметры n_estimators - управляет количеством деревьев в лесу, max_depth - максимальная глубина деревьев, min_samples_leaf - минимальное число объектов в узле.

In [ ]:
best_model = None
best_result = 0
for depth in range(1, 10):
    for est in range(5,50,5):
        for sample in range(1,5):
            model = RandomForestClassifier(random_state=12345, max_depth = depth, n_estimators=est, min_samples_leaf=sample) 
            model.fit(features_train,target_train)
            result = model.score(features_valid,target_valid)
            if result > best_result:
                best_model = model 
                best_result = result
        
        
print("Accuracy лучшей модели на валидационной выборке:", best_result)
print("max_depth:", depth, "n_estimators:", est, "min_samples_leaf:", sample)

Accuracy лучшей модели на валидационной выборке: 0.80248833592535
max_depth: 9 n_estimators: 45 min_samples_leaf: 4


Следующая модель - логистическая регрессия. Переберем гиперпараметры: регуляризацию (С), penalty и solver.

In [ ]:
logmodel = LogisticRegression(random_state=12345)

In [ ]:
parameters_grid = [
    {'penalty':['l2'],
     'C': [0.1,1,10,100],
     'solver':['lbfgs','newton-cg','liblinear','sag','saga'],
     'max_iter':[100,500]
    }
]

In [ ]:
clf = GridSearchCV(logmodel, param_grid = parameters_grid, cv = 3)

In [ ]:
train_clf = clf.fit(features_train,target_train)
best_result = clf.score(features_valid,target_valid)

print("Accuracy лучшей модели на валидационной выборке:", best_result)


Accuracy лучшей модели на валидационной выборке: 0.7589424572317263


Из проведенного исследования моделей получили результаты:
- accuracy дерева решений: 0.7884914463452566;
- accuracy случайного леса: 0.80248833592535;
- accuracy логистической регрессии: 0.7589424572317263. 

Таким образом, наилучшая модель - "Случайный лес" с параметрами:
- Depth: 9 
- N_estimators: 45 
- Min_samples_leaf: 4

В следующем шаге проверим модель на тестовой выборке.

## Проверьте модель на тестовой выборке

In [ ]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [ ]:
model = RandomForestClassifier(random_state=12345, max_depth = 9, n_estimators=45, min_samples_leaf=4) 
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Accuracy:", accuracy)

Accuracy: 0.7900466562986003


При проверке модели на тестовой выборке получили результат accuracy: 0.7900466562986003.

В данном шаге необходимо проверить модели на вменяемость, для этого используем Dummy Classifier.

## Проверьте модели на адекватность

In [ ]:
dc = DummyClassifier(random_state = 12345, strategy = "most_frequent")
dc.fit(features_train, target_train)
accuracy = dc.score(features_test, target_test)
print("Accuracy:", accuracy)

Accuracy: 0.6889580093312597


У всех моделей accuracy был выше, следовательно модели вменяемы.